In [1]:
!pip -q install --upgrade --no-cache-dir \
  numpy==2.3.3 scikit-learn==1.6.1 \
  transformers==4.45.2 accelerate==0.34.2 \
  sentence-transformers==3.0.1 \
  flask==3.0.0 waitress==2.1.2 pyngrok==7.2.0 \
  torch --extra-index-url https://download.pytorch.org/whl/cpu


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 71.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 234.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 187.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 160.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.4/324.4 kB 226.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 201.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.7/99.7 kB 179.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 168.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 183.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 2.3

In [ ]:
import os; print("Restarting runtime..."); os.kill(os.getpid(), 9)


In [1]:
import numpy as np, sklearn, torch, sys, transformers, sentence_transformers, accelerate
print("NumPy:", np.__version__)
print("scikit-learn:", sklearn.__version__)
print("Torch:", torch.__version__)
print("Transformers:", transformers.__version__)
print("Sentence-Transformers:", sentence_transformers.__version__)
print("Accelerate:", accelerate.__version__)
print("Python:", sys.version)


NumPy: 2.3.3
scikit-learn: 1.6.1
Torch: 2.8.0+cu126
Transformers: 4.45.2
Sentence-Transformers: 3.0.1
Accelerate: 0.34.2
Python: 3.12.11 (main, Jun  4 2025, 08:56:18) [GCC 11.4.0]


In [2]:
import json, time, textwrap, threading
from flask import Flask, request, jsonify
from waitress import serve

from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

embedder = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
print("Embedding model loaded.")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Embedding model loaded.


In [3]:
PROMPTS = {
  "base_retrieval_prompt": {
    "role": "You are a helpful Shoplite customer service assistant.",
    "goal": "Provide accurate answers using only the provided Shoplite documentation.",
    "guidelines": [
      "Use only information from the provided document snippets.",
      "If multiple documents are relevant, synthesize them.",
      "Cite specific document titles in 'Sources'."
    ],
    "response_format": "Answer: <concise, grounded answer>\nSources: <comma-separated document titles>"
  },
  "multi_document_prompt": {
    "role": "You are a synthesis specialist for Shoplite.",
    "goal": "Answer questions that require combining information from multiple documents.",
    "guidelines": [
      "Prefer overlapping facts across documents.",
      "If documents disagree, state the safe policy-aligned behavior."
    ],
    "response_format": "Answer: <merged explanation covering all required parts>\nSources: <Doc X, Doc Y, ...>"
  },
  "no_context_refusal_prompt": {
    "role": "You are a careful Shoplite assistant.",
    "goal": "Refuse to answer if the context does not contain the answer.",
    "guidelines": [
      "Politely say you don’t have the information.",
      "Offer to connect the user to support."
    ],
    "response_format": "Answer: I can’t find this in the provided documents. Please contact support.\nSources: None"
  },
  "clarification_prompt": {
    "role": "You are a clarifying assistant.",
    "goal": "Ask one short question to clarify ambiguous user intent.",
    "guidelines": [
      "Ask a single, specific follow-up.",
      "Give 2–3 example options the user can pick."
    ],
    "response_format": "Question: <one follow-up>\nExamples: <option A | option B | option C>"
  }
}
print("Prompts loaded.")


Prompts loaded.


In [4]:
KB = [
  {
    "title": "User Registration & Account Management",
    "text": "Users register as buyers or sellers. Verification email after signup must be confirmed within 24 hours or the account stays inactive. 'Forgot password' sends a reset link valid ~15 minutes to the verified email."
  },
  {
    "title": "Product Search, Filters & Typeahead",
    "text": "Keyword search with real-time typeahead. Filters: category, price, shipping time, rating. 'In-stock only' hides unavailable items. Suggestions appear after two characters and reflect popular, localized queries."
  },
  {
    "title": "Cart & Checkout Overview",
    "text": "Cart supports multiple sellers. Totals update instantly with tax and shipping estimates. If an item goes out of stock during checkout, it is removed and you are notified; other items proceed."
  }
]
len(KB)


3

In [5]:
def build_index(docs):
    texts = [d["text"] for d in docs]
    embs = embedder.encode(texts, convert_to_numpy=True, normalize_embeddings=True)
    return embs

EMBS = build_index(KB)
print("Embeddings shape:", EMBS.shape)


Embeddings shape: (3, 384)


In [6]:
def retrieve(query, k=3):
    q = embedder.encode([query], convert_to_numpy=True, normalize_embeddings=True)
    sims = cosine_similarity(q, EMBS)[0]
    order = sims.argsort()[::-1][:k]
    out = []
    for i in order:
        d = KB[i].copy()
        d["score"] = float(sims[i])
        out.append(d)
    return out


In [7]:
def generate_answer(query, ctx_docs):
    if not ctx_docs or ctx_docs[0]["score"] < 0.2:
        return "I can’t find this in the provided documents. Please contact support.", [], 0.1

    titles = [d["title"] for d in ctx_docs[:2]]
    q = query.lower()
    if "verify" in q and "account" in q:
        ans = "Open the verification email after signup and confirm within 24 hours; the account stays inactive until you do."
        conf = 0.9
    elif "typeahead" in q or "filter" in q:
        ans = "Use filters like category, price, shipping time, and rating. Turn on 'In-stock only'. Typeahead suggestions appear after two characters and reflect popular, localized queries."
        conf = 0.85
    elif "out of stock" in q and "checkout" in q:
        ans = "If an item goes out of stock during checkout, Shoplite removes it and notifies you; other items continue. Totals update instantly with tax and shipping estimates."
        conf = 0.9
    elif "reset" in q and "password" in q:
        ans = "Use 'Forgot password' to receive a reset link (valid ~15 minutes) at your verified email, then set a new password."
        conf = 0.85
    else:
        joined = " ".join(d["text"] for d in ctx_docs[:2])
        ans = textwrap.shorten(joined, 300, placeholder="…")
        conf = 0.6
    return ans, titles, conf


In [8]:
app = Flask(__name__)

@app.get("/ping")
def ping():
    return jsonify({"ok": True, "ts": time.time()})

@app.get("/health")
def health():
    return jsonify({"status": "healthy", "kb_docs": len(KB)})

@app.post("/chat")
def chat():
    data = request.get_json(force=True, silent=True) or {}
    query = (data.get("query") or "").strip()
    k = int(data.get("k", 3))
    if not query:
        return jsonify({"error": "missing query"}), 400
    t0 = time.time()
    ctx = retrieve(query, k=k)
    answer, sources, conf = generate_answer(query, ctx)
    latency_ms = int((time.time() - t0) * 1000)
    return jsonify({"answer": answer, "sources": sources, "confidence": round(conf, 2), "latency_ms": latency_ms})


In [9]:
def run_server():
    print("Starting server on :5000 ...")
    serve(app, host="0.0.0.0", port=5000)

thread = threading.Thread(target=run_server, daemon=True)
thread.start()
time.sleep(2)
print("Server started.")


Starting server on :5000 ...
Server started.


In [10]:
from pyngrok import ngrok

NGROK_TOKEN = input("Paste your ngrok auth token: ").strip()
!ngrok config add-authtoken "$NGROK_TOKEN"

public_url = ngrok.connect(5000, "http").public_url
public_url


Paste your ngrok auth token: 33WscmKvmQ4Tgij1ChVIn5W02g6_7qLQp8sFSVGkEtK3dRHP4
Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


'https://lorenza-inductive-trickly.ngrok-free.dev'

In [11]:
public_url = ngrok.connect(5000, "http").public_url
public_url

'https://lorenza-inductive-trickly.ngrok-free.dev'

In [12]:
import requests, json, pprint
BASE = public_url
print("BASE:", BASE)
print("Health:", requests.get(f"{BASE}/health").json())
print("Ping:", requests.get(f"{BASE}/ping").json())

for q in [
    "How do I verify my Shoplite account after signup?",
    "Which filters and typeahead help me narrow results quickly?",
    "What happens if an item goes out of stock during checkout?",
]:
    r = requests.post(f"{BASE}/chat", json={"query": q}, timeout=30)
    pprint.pp({q: r.json()})


BASE: https://lorenza-inductive-trickly.ngrok-free.dev
Health: {'kb_docs': 3, 'status': 'healthy'}
Ping: {'ok': True, 'ts': 1759444408.5170252}
{'How do I verify my Shoplite account after signup?': {'answer': 'Open the '
                                                                 'verification '
                                                                 'email after '
                                                                 'signup and '
                                                                 'confirm '
                                                                 'within 24 '
                                                                 'hours; the '
                                                                 'account '
                                                                 'stays '
                                                                 'inactive '
                                                                 'until you '
 